In [1]:
pip install xgboost lightgbm catboost optuna

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 3.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 8.0 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 kB 5.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 21.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt; plt.style.use('ggplot')
import seaborn as sns

from math import floor

from scipy.stats import rankdata
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

import optuna as op

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-4/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-4/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-4/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

# train['Time'] = train['Time'] / 3600
# test['Time'] = test['Time'] / 3600

# train = train[(train['Time'] >= 10) & (train['Time'] <= 24)].reset_index(drop = True)
# train['Time_Bucket'] = train['Time'].apply(floor)

# train['Time_Bucket'] = train['Time_Bucket'] - 8

# test['Time_Bucket'] = test['Time'].apply(floor)
# test['Time_Bucket'] = test['Time_Bucket'] - 32

In [11]:
test.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,219129,120580.0,2.115519,-0.691809,-1.305514,-0.685655,-0.641265,-0.764784,-0.924262,-0.023030,-0.230126,0.220610,1.058325,-0.723363,-0.619241,-2.243711,-0.271429,2.007845,1.213534,0.282344,0.579121,0.067367,0.241708,0.682524,0.037769,-0.546859,-0.123055,-0.084889,0.004720,-0.021944,29.95
1,219130,120580.0,1.743525,-1.681429,-0.547387,-1.061113,-0.695825,2.458824,-1.632859,1.073529,1.068183,0.483337,0.274519,-0.346514,-0.409519,-0.564673,-0.143855,1.334855,0.160901,-0.087356,-0.066264,0.441788,0.543278,1.294571,0.309541,3.703925,-0.242579,0.068708,0.002629,0.064690,163.50
2,219131,120580.0,2.205568,-1.571445,-0.238965,-1.662517,-1.652324,-0.054701,-1.682064,0.105613,-1.177858,1.626352,0.601148,-0.041610,0.432748,-0.527684,-0.543665,0.043896,0.058472,0.622667,0.010671,-0.366906,-0.131527,0.086623,0.291375,0.739087,-0.543006,-0.297813,0.043699,-0.037855,16.00
3,219132,120580.0,1.989728,-0.972909,-1.938259,-1.440129,-0.166855,-0.794048,0.252889,-0.399789,2.079398,-1.225592,-0.869368,1.647638,1.138026,-0.258468,-0.361765,-1.253326,0.099479,-0.587702,1.207085,-0.049136,-0.080115,-0.010732,-0.038550,0.656830,0.343470,-0.627529,-0.024338,-0.036143,120.98
4,219133,120580.0,-1.943548,-1.668761,0.363601,-0.977610,2.684779,-2.037681,0.039709,-0.048895,-0.281749,-0.341879,0.471628,0.637604,-0.592416,0.331702,-1.451799,0.096467,-0.853439,-0.462060,-0.267098,0.391627,0.083389,-0.306918,0.247822,-0.391799,-0.790716,-0.025706,0.330758,0.335537,1.98


In [10]:
test[test.drop(columns = ['id', 'Time', 'Amount'], axis = 1).duplicated()]

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
31183,250312,131086.0,2.040211,-0.146975,-2.955934,-0.578356,2.609358,3.142642,-0.416883,0.784393,0.359902,-0.351075,0.329651,0.183508,-0.272919,-0.597437,0.583897,0.178676,0.473898,-0.498850,-0.140099,-0.120714,-0.352334,-0.996937,0.363485,0.604827,-0.264560,0.219671,-0.039209,-0.042787,1.98
52482,271611,137900.0,2.040211,-0.146975,-2.955934,-0.578356,2.609358,3.142642,-0.416883,0.784393,0.359902,-0.351075,0.329651,0.183508,-0.272919,-0.597437,0.583897,0.178676,0.473898,-0.498850,-0.140099,-0.120714,-0.352334,-0.996937,0.363485,0.604827,-0.264560,0.219671,-0.039209,-0.042787,1.98
56607,275736,139821.0,2.040211,-0.146975,-2.955934,-0.578356,2.609358,3.142642,-0.416883,0.784393,0.359902,-0.351075,0.329651,0.183508,-0.272919,-0.597437,0.583897,0.178676,0.473898,-0.498850,-0.140099,-0.120714,-0.352334,-0.996937,0.363485,0.604827,-0.264560,0.219671,-0.039209,-0.042787,1.98
66110,285239,142512.0,2.085175,0.393051,-4.508201,-0.311771,3.510117,2.453299,0.220469,0.543377,-0.100434,-1.018622,0.807038,-0.330548,-0.531419,-2.181449,0.388724,0.232071,2.125022,0.405549,-0.440930,-0.086989,-0.067217,-0.072642,-0.036584,0.529693,0.414685,0.735870,-0.058233,-0.026658,0.76
66418,285547,142564.0,2.085175,0.393051,-4.508201,-0.311771,3.510117,2.453299,0.220469,0.543377,-0.100434,-1.018622,0.807038,-0.330548,-0.531419,-2.181449,0.388724,0.232071,2.125022,0.405549,-0.440930,-0.086989,-0.067217,-0.072642,-0.036584,0.529693,0.414685,0.735870,-0.058233,-0.026658,0.76
66678,285807,142611.0,2.048629,-0.367489,-2.544065,-0.728472,2.387380,3.318407,-0.477692,0.791276,0.403246,0.092725,0.004406,0.390211,-0.212361,0.495790,0.472770,-0.171451,-0.368228,-0.976348,0.022521,-0.152758,-0.292703,-0.835377,0.380780,0.698235,-0.265199,0.219159,-0.052337,-0.066750,1.98
72343,291472,144414.0,2.040211,-0.146975,-2.955934,-0.578356,2.609358,3.142642,-0.416883,0.784393,0.359902,-0.351075,0.329651,0.183508,-0.272919,-0.597437,0.583897,0.178676,0.473898,-0.498850,-0.140099,-0.120714,-0.352334,-0.996937,0.363485,0.604827,-0.264560,0.219671,-0.039209,-0.042787,1.98
76659,295788,145657.0,2.108108,-0.207740,-3.386067,-0.720760,2.905339,2.932171,0.054796,0.562129,0.017654,0.190507,-0.079089,0.232614,-0.366430,0.797040,0.085960,-0.721847,-0.169346,-0.895393,0.002128,-0.174293,0.095248,0.367526,0.010535,0.784185,0.412944,0.734475,-0.093999,-0.091945,0.76
80516,299645,146754.0,2.085175,0.393051,-4.508201,-0.311771,3.510117,2.453299,0.220469,0.543377,-0.100434,-1.018622,0.807038,-0.330548,-0.531419,-2.181449,0.388724,0.232071,2.125022,0.405549,-0.440930,-0.086989,-0.067217,-0.072642,-0.036584,0.529693,0.414685,0.735870,-0.058233,-0.026658,0.76
85443,304572,148310.0,2.040211,-0.146975,-2.955934,-0.578356,2.609358,3.142642,-0.416883,0.784393,0.359902,-0.351075,0.329651,0.183508,-0.272919,-0.597437,0.583897,0.178676,0.473898,-0.498850,-0.140099,-0.120714,-0.352334,-0.996937,0.363485,0.604827,-0.264560,0.219671,-0.039209,-0.042787,1.98


In [7]:
train[train.drop(columns = ['id', 'Time', 'Amount', 'Class'], axis = 1).duplicated(keep = 'first')]

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
5689,5689,3027.0,-0.302521,0.447474,-0.495757,-3.214799,2.705041,2.762440,0.593847,0.556451,0.697720,-1.090181,-0.111088,0.062652,-0.238457,0.215738,1.069356,-0.255231,-1.072085,0.485110,0.271315,0.156089,0.007343,0.250886,-0.331600,1.009952,0.233730,-0.824471,0.241305,-0.020359,1.00,0
5722,5722,3088.0,-0.302521,0.447474,-0.495757,-3.214799,2.705041,2.762440,0.593847,0.556451,0.697720,-1.090181,-0.111088,0.062652,-0.238457,0.215738,1.069356,-0.255231,-1.072085,0.485110,0.271315,0.156089,0.007343,0.250886,-0.331600,1.009952,0.233730,-0.824471,0.241305,-0.020359,1.00,0
5728,5728,3094.0,-0.302521,0.447474,-0.495757,-3.214799,2.705041,2.762440,0.593847,0.556451,0.697720,-1.090181,-0.111088,0.062652,-0.238457,0.215738,1.069356,-0.255231,-1.072085,0.485110,0.271315,0.156089,0.007343,0.250886,-0.331600,1.009952,0.233730,-0.824471,0.241305,-0.020359,1.00,0
17492,17492,26796.0,1.245674,0.166975,0.488306,0.635322,-0.562777,-1.011073,0.014953,-0.160211,0.170362,-0.044575,-0.356749,-0.073460,-0.517760,0.406969,1.124147,0.342470,-0.374656,-0.438992,-0.116091,-0.132080,-0.262581,-0.816264,0.140304,0.357827,0.186423,0.096544,-0.035866,0.018495,8.99,0
18035,18035,27604.0,1.245674,0.166975,0.488306,0.635322,-0.562777,-1.011073,0.014953,-0.160211,0.170362,-0.044575,-0.356749,-0.073460,-0.517760,0.406969,1.124147,0.342470,-0.374656,-0.438992,-0.116091,-0.132080,-0.262581,-0.816264,0.140304,0.357827,0.186423,0.096544,-0.035866,0.018495,8.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208693,208693,113549.0,2.055797,-0.326668,-2.752041,-0.842316,2.463072,3.173856,-0.432126,0.727706,0.608606,-0.075186,0.063504,0.350564,-0.141238,0.690972,1.275257,-0.371962,-0.601957,-0.052640,-0.330590,-0.180370,0.269765,0.844627,0.020675,0.726212,0.366624,-0.398828,0.027735,-0.060282,1.00,0
208698,208698,113550.0,2.055797,-0.326668,-2.752041,-0.842316,2.463072,3.173856,-0.432126,0.727706,0.608606,-0.075186,0.063504,0.350564,-0.141238,0.690972,1.275257,-0.371962,-0.601957,-0.052640,-0.330590,-0.180370,0.269765,0.844627,0.020675,0.726212,0.366624,-0.398828,0.027735,-0.060282,1.00,0
208722,208722,113573.0,2.055797,-0.326668,-2.752041,-0.842316,2.463072,3.173856,-0.432126,0.727706,0.608606,-0.075186,0.063504,0.350564,-0.141238,0.690972,1.275257,-0.371962,-0.601957,-0.052640,-0.330590,-0.180370,0.269765,0.844627,0.020675,0.726212,0.366624,-0.398828,0.027735,-0.060282,1.00,0
208726,208726,113577.0,2.055797,-0.326668,-2.752041,-0.842316,2.463072,3.173856,-0.432126,0.727706,0.608606,-0.075186,0.063504,0.350564,-0.141238,0.690972,1.275257,-0.371962,-0.601957,-0.052640,-0.330590,-0.180370,0.269765,0.844627,0.020675,0.726212,0.366624,-0.398828,0.027735,-0.060282,1.00,0


In [8]:
train[train.drop(columns = ['id', 'Time', 'Amount', 'Class'], axis = 1).duplicated(keep = 'first')]['Class'].value_counts()

0    95
Name: Class, dtype: int64

In [37]:
def getDuplicateColumns(df):

    duplicatedColumnNames = set()
    
    for x in range(df.shape[1]):
        
        col = df.iloc[:, x]
        
        for y in range(x + 1, df.shape[1]):
            
            otherCol = df.iloc[:, y]
            
            if col.equals(otherCol):
                
                duplicateColumnNames.add(df.columns.values[y])
    
    return list(duplicatedColumnNames)

In [38]:
getDuplicateColumns(train)

[]

In [33]:
def getDuplicateColumns(df):
    
    # Create an empty set
    duplicateColumnNames = list()
    
    # Iterate through all the columns
    # of dataframe
    for x in range(df.shape[1]):
     
        # Take column at xth index.
        col = df.iloc[:, x]
 
        # Iterate through all the columns in
        # DataFrame from (x + 1)th index to
        # last index
        for y in range(x + 1, df.shape[1]):
 
            # Take column at yth index.
            otherCol = df.iloc[:, y]
 
            # Check if two columns at x & y
            # index are equal or not,
            # if equal then adding
            # to the set
            if col.equals(otherCol):
                
                duplicateColumnNames.append(df.columns.values[y])
 
    # Return list of unique column names
    # whose contents are duplicates.
    return list(duplicateColumnNames)

SyntaxError: invalid non-printable character U+00A0 (1293650629.py, line 12)

In [6]:
sum(train.duplicated())

0

In [3]:
data_temp = pd.DataFrame(train.drop(columns = ['id', 'Time', 'Amount', 'Class'], axis = 1).groupby('Time_Bucket').mean())
data_temp['Time_Bucket'] = data_temp.index
data_temp = data_temp.reset_index(drop = True)

col_names = ['var_agg_' + str(i) for i in range(1, 29)]
col_names.append('Time_Bucket')
data_temp.columns = col_names
data_temp

,var_agg_1,var_agg_2,var_agg_3,var_agg_4,var_agg_5,var_agg_6,var_agg_7,var_agg_8,var_agg_9,var_agg_10,var_agg_11,var_agg_12,var_agg_13,var_agg_14,var_agg_15,var_agg_16,var_agg_17,var_agg_18,var_agg_19,var_agg_20,var_agg_21,var_agg_22,var_agg_23,var_agg_24,var_agg_25,var_agg_26,var_agg_27,var_agg_28,Time_Bucket
0,0.333730,-0.073256,0.727621,0.193368,-0.342040,-0.003679,-0.149246,0.111034,-0.208534,0.051404,0.184182,0.183007,-0.200466,-0.026943,0.261037,0.084592,-0.044923,-0.166605,-0.011051,0.017965,-0.026261,-0.102430,-0.044435,0.043686,0.203128,0.019098,0.014827,0.023276,10
1,0.201092,-0.050582,0.680033,0.212614,-0.388415,-0.004704,-0.184821,0.157824,-0.191671,0.033779,0.181890,0.147877,-0.275214,-0.026553,0.226033,0.080437,-0.045081,-0.175022,0.006734,0.024437,-0.025395,-0.091800,-0.046984,0.048612,0.194880,0.018525,0.013343,0.018880,11
2,0.045770,-0.143465,0.749928,0.034679,-0.322463,0.212409,-0.180764,0.180882,-0.312830,0.078715,0.102855,0.131944,-0.156471,-0.086651,0.160381,-0.008999,0.011185,-0.055712,0.014463,0.040521,-0.011739,-0.041531,-0.068954,-0.024154,0.170074,0.018252,0.005989,0.020543,12
3,0.045649,-0.062080,0.740161,0.098806,-0.256183,0.242485,-0.181309,0.191641,-0.229642,-0.003426,0.082685,0.142292,-0.226008,-0.094263,0.138450,0.021897,0.026282,-0.083011,0.026370,0.027055,-0.011089,-0.041326,-0.079701,-0.043637,0.187188,0.005667,0.012436,0.019209,13
4,-0.035857,-0.107233,0.768588,0.068035,-0.299196,0.218773,-0.194045,0.201716,-0.257216,0.033322,0.103942,0.147191,-0.220203,-0.074356,0.146401,-0.018432,0.026805,-0.066279,-0.005564,0.027664,-0.014945,-0.033958,-0.066026,-0.021528,0.158207,0.026648,0.009694,0.018271,14
5,-0.041764,-0.100591,0.748202,0.062950,-0.300529,0.191367,-0.186521,0.181876,-0.263160,0.027804,0.105258,0.148332,-0.209786,-0.072754,0.161212,-0.015826,0.036743,-0.074342,-0.010203,0.034587,-0.011062,-0.040649,-0.066872,-0.017182,0.158374,0.034891,0.007992,0.022306,15
6,-0.082209,-0.078978,0.768831,0.096932,-0.260186,0.217701,-0.169631,0.193696,-0.257516,0.018590,0.083418,0.157368,-0.188094,-0.075249,0.179256,-0.017999,0.037062,-0.079835,-0.008361,0.041754,-0.011645,-0.051519,-0.065988,-0.002150,0.150515,0.028634,0.009558,0.020044,16
7,-0.023450,0.006944,0.705619,0.113274,-0.216285,0.258059,-0.162874,0.211594,-0.265264,0.004382,0.099685,0.182501,-0.156276,-0.056883,0.226645,-0.017461,0.013288,-0.035378,-0.001541,0.036509,-0.023347,-0.065629,-0.069894,-0.016096,0.157499,0.017445,0.015741,0.021650,17
8,-0.021863,-0.016917,0.837527,0.062716,-0.280615,0.325233,-0.246398,0.215462,-0.182669,0.007972,0.028695,0.156635,-0.201825,-0.153261,0.076565,0.018011,0.054876,-0.101032,0.049292,0.019329,-0.010886,-0.008126,-0.072063,-0.019760,0.174362,0.005380,0.024530,0.020539,18
9,-0.042821,0.106643,0.714383,0.103158,-0.181231,0.208391,-0.135898,0.196532,-0.286955,0.008178,0.079471,0.194962,-0.118301,-0.061878,0.234306,-0.034075,-0.003813,-0.009115,-0.010561,0.020150,-0.052812,-0.108194,-0.061349,-0.008682,0.143373,0.030707,0.014660,0.023993,19


In [4]:
train_agg = pd.merge(train, data_temp, on = 'Time_Bucket', how = 'left')
train_agg

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class,Time_Bucket,var_agg_1,var_agg_2,var_agg_3,var_agg_4,var_agg_5,var_agg_6,var_agg_7,var_agg_8,var_agg_9,var_agg_10,var_agg_11,var_agg_12,var_agg_13,var_agg_14,var_agg_15,var_agg_16,var_agg_17,var_agg_18,var_agg_19,var_agg_20,var_agg_21,var_agg_22,var_agg_23,var_agg_24,var_agg_25,var_agg_26,var_agg_27,var_agg_28
0,28639,10.002500,1.240257,-0.064968,0.292301,0.562824,-0.463260,-0.643769,-0.290874,0.074817,0.520861,-0.059493,-0.401788,-0.906394,-2.191959,0.763318,1.537556,0.055868,-0.018766,-0.812256,-0.466929,-0.270077,-0.269914,-0.804726,0.184878,-0.017442,0.066157,0.139930,-0.032306,0.005789,1.98,0,10,0.333730,-0.073256,0.727621,0.193368,-0.342040,-0.003679,-0.149246,0.111034,-0.208534,0.051404,0.184182,0.183007,-0.200466,-0.026943,0.261037,0.084592,-0.044923,-0.166605,-0.011051,0.017965,-0.026261,-0.102430,-0.044435,0.043686,0.203128,0.019098,0.014827,0.023276
1,28640,10.002500,0.566687,-1.065377,0.843466,1.347078,-1.132464,0.499793,-0.466450,0.325933,0.565898,-0.102979,1.255589,0.441395,-1.050434,0.354097,1.148959,0.492468,-0.520868,0.638387,-0.719238,0.420869,0.390013,0.530061,-0.381049,0.027772,0.228536,-0.263094,0.024399,0.072729,295.20,0,10,0.333730,-0.073256,0.727621,0.193368,-0.342040,-0.003679,-0.149246,0.111034,-0.208534,0.051404,0.184182,0.183007,-0.200466,-0.026943,0.261037,0.084592,-0.044923,-0.166605,-0.011051,0.017965,-0.026261,-0.102430,-0.044435,0.043686,0.203128,0.019098,0.014827,0.023276
2,28641,10.002500,-0.666978,0.582365,1.320170,1.038262,-0.717517,-0.698978,-0.358901,0.562265,0.418491,-0.639646,-0.848472,-0.099086,-1.336930,0.183388,-0.006867,-0.326828,0.329010,-0.116537,0.443480,-0.230117,-0.022416,-0.054777,0.161706,0.737483,-1.037992,-0.385097,0.128122,0.143395,5.75,0,10,0.333730,-0.073256,0.727621,0.193368,-0.342040,-0.003679,-0.149246,0.111034,-0.208534,0.051404,0.184182,0.183007,-0.200466,-0.026943,0.261037,0.084592,-0.044923,-0.166605,-0.011051,0.017965,-0.026261,-0.102430,-0.044435,0.043686,0.203128,0.019098,0.014827,0.023276
3,28642,10.002500,0.981675,-0.207519,1.261230,1.510517,-1.050243,-0.001808,-0.492566,0.126880,0.865598,-0.261810,-0.594468,0.624652,0.113990,-0.402636,0.186490,-0.016023,-0.056938,-0.543408,-0.397957,-0.025256,-0.037674,0.013838,-0.038693,0.443485,0.383349,-0.429776,0.064349,0.051706,74.99,0,10,0.333730,-0.073256,0.727621,0.193368,-0.342040,-0.003679,-0.149246,0.111034,-0.208534,0.051404,0.184182,0.183007,-0.200466,-0.026943,0.261037,0.084592,-0.044923,-0.166605,-0.011051,0.017965,-0.026261,-0.102430,-0.044435,0.043686,0.203128,0.019098,0.014827,0.023276
4,28643,10.002500,1.152601,0.181725,1.235559,1.073024,-0.812595,-0.337824,-0.438645,0.043215,0.373339,-0.047564,-0.230448,0.547962,0.602038,-0.065167,1.427969,0.368845,-0.505403,-0.160750,-0.886036,-0.112488,0.044444,0.205976,0.062316,0.390864,0.270659,-0.421760,0.065232,0.036674,0.12,0,10,0.333730,-0.073256,0.727621,0.193368,-0.342040,-0.003679,-0.149246,0.111034,-0.208534,0.051404,0.184182,0.183007,-0.200466,-0.026943,0.261037,0.084592,-0.044923,-0.166605,-0.011051,0.017965,-0.026261,-0.102430,-0.044435,0.043686,0.203128,0.019098,0.014827,0.023276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
164301,192940,23.996111,-0.502437,1.368014,1.170955,-0.069564,0.341435,-0.746616,0.743426,-0.068899,-0.715814,-0.424011,-0.269254,0.562312,1.671339,-0.752812,0.806948,0.434965,-0.037238,0.172737,0.041817,0.225974,-0.248877,-0.644470,-0.091666,0.003145,-0.167749,0.053969,0.120813,0.161414,1.29,0,23,-0.270020,0.289964,0.782376,0.003799,-0.095494,0.260937,-0.092785,0.238086,-0.088260,-0.146722,0.044082,0.313698,-0.065349,-0.046500,0.275114,-0.044054,-0.071262,0.027703,0.060313,0.037259,-0.045961,-0.089739,-0.067466,-0.

# XGBoost Modeling

In [8]:
# to_select = ['var_agg_' + str(i) for i in range(1, 29)]
# to_select.append('Amount')
X = train_agg.drop(columns = ['id', 'Time', 'Time_Bucket', 'Class'])
Y = train_agg['Class']
# test = test.drop(columns = ['id', 'Time'], axis = 1)

XGB_cv_scores, XGB_imp = list(), list()
preds = list()

## Running 5 times CV
for i in range(5):
    
    skf = StratifiedKFold(n_splits = 5, shuffle = False)
    
    for train_ix, test_ix in skf.split(X, Y):
        
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
                
        ## Building RF model
        XGB_md = XGBClassifier(tree_method = 'hist',
                               colsample_bytree = 0.7, 
                               gamma = 0.72, 
                               learning_rate = 0.01, 
                               max_depth = 7, 
                               min_child_weight = 10, 
                               n_estimators = 1000, 
                               subsample = 0.7).fit(X_train, Y_train)
        XGB_imp.append(XGB_md.feature_importances_)
        
        ## Predicting on X_test and test
        XGB_pred_1 = XGB_md.predict_proba(X_test)[:, 1]
#         XGB_pred_2 = XGB_md.predict_proba(test)[:, 1]
        
        ## Computing roc-auc score
        XGB_cv_scores.append(roc_auc_score(Y_test, XGB_pred_1))
#         preds.append(XGB_pred_2)

XGB_cv_score = np.mean(XGB_cv_scores)    
print('The average roc-auc score over 5-folds (run 5 times) is:', XGB_cv_score)

The average roc-auc score over 5-folds (run 5 times) is: 0.7507637208975015


In [ ]:
plt.figure(figsize = (12, 14))
pd.DataFrame(XGB_imp, columns = X.columns).apply(np.mean, axis = 0).sort_values().plot(kind = 'barh');
plt.xlabel('XGBoost Score')
plt.ylabel('Feature')
plt.show();